In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

In [ ]:
url = "https://raw.githubusercontent.com/tofrie03/ML_Assignment/refs/heads/main/Material/Energydata_Spain_202304-202504.csv"

df = pd.read_csv(url)

Data Cleaning

In [ ]:
try:
    print(df)

    # Prüfe die Datentypen aller Spalten
    print("Datentypen aller Spalten:\n", df.dtypes)

    # Auf fehlende Werte prüfen
    missing = df.isnull().sum()
    print("Fehlende Werte pro Spalte:\n", missing)

    # Keine fehlenden Werte gefunden

    # Duplikate indentifizieren und entfernen
    duplicates = df.duplicated().sum()
    print(f"Anzahl Duplikate: {duplicates}")

    df.drop_duplicates(inplace=True)

    print("Datenbereinigung abgeschlossen.")

except Exception as e:
    print("Fehler bei der Datenbereinigung:", e)

Data Visualization

In [ ]:
try:
    # datetime-Spalte in datetime-Objekte umwandeln
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Zeitreihe der Energiequellen anzeigen
    energy_sources = ['Wind', 'Nuclear', 'Coal', 'Solar PV', 'Hydro']

    plt.figure(figsize=(14, 6))
    for src in energy_sources:
        plt.plot(df['datetime'], df[src], label=src, alpha=0.7)
    plt.title("Zeitverlauf der Energieerzeugung")
    plt.xlabel("Datum")
    plt.ylabel("Leistung (MW)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Korrelationsmatrix aller numerischen Spalten
    numeric_df = df.select_dtypes(include='number')
    corr = numeric_df.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
    plt.title("Korrelationsmatrix der numerischen Merkmale")
    plt.tight_layout()
    plt.show()

    # Auswahl der Spalten für das Pairplot
    pairplot_columns = ['value', 'Wind', 'Nuclear', 'Coal', 'Solar PV', 'day_period']
    g = sns.pairplot(df[pairplot_columns], hue='day_period', palette='Set2', height=1.8, aspect=1.2)
    g.fig.suptitle("Pairplot zur Analyse von Attributbeziehungen", y=1.02)

    # Legende rechts außen positionieren
    g._legend.set_bbox_to_anchor((1.05, 0.5))
    g._legend.set_loc("center left")

    plt.tight_layout()
    plt.show()

    # Boxplot: Verteilung des Strompreises nach Tageszeit
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=df, x='day_period', y='value', palette='Set2')
    plt.title("Verteilung des Strompreises nach Tageszeit")
    plt.ylabel("Strompreis (€/MWh)")
    plt.xlabel("Tageszeit")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Scatterplot mit Schwellenwertlinie
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='Wind', y='value', alpha=0.6)
    plt.axhline(y=50, color='red', linestyle='--', label='Preisgrenze: 50 €/MWh')
    plt.title("Windleistung vs. Strompreis mit Preisgrenze")
    plt.xlabel("Windleistung (MW)")
    plt.ylabel("Strompreis (€/MWh)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Histogramm der Strompreise
    plt.figure(figsize=(10, 5))
    sns.histplot(df['value'], bins=50, kde=True, color='skyblue')
    plt.title("Verteilung der Strompreise")
    plt.xlabel("Preis (€/MWh)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Strompreis über die gesamte Zeit – alle Einzelwerte
    plt.figure(figsize=(14, 6))
    plt.plot(df['datetime'], df['value'], linestyle='-', marker='', alpha=0.6)
    plt.title("Strompreis über die gesamte Zeit")
    plt.xlabel("Datum")
    plt.ylabel("Strompreis (€/MWh)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print(df)

except Exception as e:
    print("Fehler bei der explorativen Analyse:", e)

Data Preprocessing

In [ ]:
try:
    print(df)
    # Spalten, die nicht normalisiert werden sollen
    non_numeric_cols = ['datetime', 'day_period', 'weekday_type', 'season']
    features_to_normalize = []

    for col in df.columns:
        if col not in non_numeric_cols:
            features_to_normalize.append(col)

    # Standardisierung
    scaler = StandardScaler()
    df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])

    print(df)
        
    # Zeitfeatures aus 'datetime' extrahieren
    df['hour'] = df['datetime'].dt.hour
    df['month'] = df['datetime'].dt.month
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df.drop(columns=['datetime'], inplace=True)

    print(df)
    
    # Kategorische Spalten als numerische Labels kodieren
    categorical_cols = ['day_period', 'weekday_type', 'season']
    for col in categorical_cols:
        df[col] = df[col].astype('category').cat.codes

except Exception as e:
    print("Fehler:", e)

print(df)

Train Test Split

In [ ]:
# Train-Test-Split
X = df.drop(columns=['value'])
y = df['value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {X_train.shape}, Testdaten: {X_test.shape}")

Train Model

In [ ]:
try:
    max_depth = 50

    max_depth_range = range(1, max_depth + 1)

    train_losses = []
    test_losses = []

    for depth in max_depth_range:
        model = RandomForestRegressor(max_depth=depth, random_state=42)
        model.fit(X_train, y_train)

        train_loss = ((y_train - model.predict(X_train)) ** 2).mean()
        test_loss = ((y_test - model.predict(X_test)) ** 2).mean()

        train_losses.append(train_loss)
        test_losses.append(test_loss)

        print(f"Depth: {depth:2} \tTrain MSE: {train_loss:.3f} \tTest MSE: {test_loss:.3f}")

except Exception as e:
    print("Fehler bei der Optimierung des Random Forest:", e)

In [ ]:
try:
    # Train-Loss vs. Test-Loss
    plt.plot(max_depth_range, train_losses, label="Train Loss", marker='o')
    plt.plot(max_depth_range, test_losses, label="Test Loss", marker='o')
    plt.xlabel("Tree Depth")
    plt.ylabel("MSE")
    plt.title("Random Forest Regressor: Loss vs. Depth")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


    # Bias-Variance Tradeoff
    plt.figure(figsize=(10, 6))
    plt.plot(max_depth_range, train_losses, label="Trainingsfehler (Bias)", marker='o')
    plt.plot(max_depth_range, test_losses, label="Testfehler (Varianz)", marker='o')
    plt.axvline(x=max_depth_range[np.argmin(test_losses)], color='green', linestyle='--', label="Optimale Tiefe")
    plt.title("Bias-Variance Tradeoff")
    plt.xlabel("Baumtiefe")
    plt.ylabel("Mittlerer quadratischer Fehler (MSE)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

except Exception as e:
    print("Fehler bei der Analyse:", e)